In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
from recommender_tool import FashionOutfitGenerator
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.chains import ConversationalRetrievalChain,LLMChain
from langchain.chains.question_answering import load_qa_chain
from config import *

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

In [3]:
from langchain.prompts.prompt import PromptTemplate

_template = """
You are a friendly, conversational clothing shopping assistant.
Use the following context to recommend the outfit with product names, descriptions, and keywords to show the shopper whats available, 
help find what they want, and answer any questions. Be descriptive in your answers.
Context:
\"""
{context}
\"""
Question:\"
\"""
Helpful answer:
"""

QA_TEMPLATE = PromptTemplate.from_template(_template)

In [4]:
llm_chain = load_qa_chain(llm,verbose=True)

In [5]:
llm_chain.run({"context":"Hi, How may I assist you"})

ValueError: Missing some input keys: {'question', 'input_documents'}

In [ ]:
from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import langchain

In [ ]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.Index(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

ModuleNotFoundError: No module named 'faiss'

In [ ]:
retriever =vectorstore.as_retriever(search_kwargs={"k":10})

NameError: name 'vectorstore' is not defined

In [6]:
from langchain.vectorstores.redis import Redis

In [7]:
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY1)
vectorstore = Redis(
    redis_url="redis://@localhost:6380",
    index_name="history0",
    embedding_function=embeddings.embed_query,
)
vectorstore._create_index()
vectorstore.add_texts(["hello world", "hello world 2"])

NameError: name 'OpenAIEmbeddings' is not defined

In [147]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs=dict(k=1))

In [148]:
memory = VectorStoreRetrieverMemory(retriever=retriever,memory_key="history")

In [149]:
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)

In [150]:
llm = ChatOpenAI(temperature=0)
llmchain = ConversationChain(llm=llm,memory=memory,verbose=True,prompt=PROMPT)

In [151]:
llmchain.predict(input="Hi, Suggest me an outfit for a party.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi
response: Hello! How can I assist you today?
input: What is my age ?
response: Your age is 12.
input: What is my age ?
response: Your age is 12.
input: What is my age ?
response: Your age is 12.

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, Suggest me an outfit for a party.
AI:

> Finished chain.


"Hello! I'd be happy to help you find an outfit for a party. Could you please provide me with some more information? What is the dress code for the party? Is there a specific theme or color scheme?"

In [153]:
llmchain.predict(input="The party is a summer swimming pool party.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, Suggest me an outfit for a party.
response: Hello! I'd be happy to help you find an outfit for a party. Could you please provide me with some more information? What is the dress code for the party? Is there a specific theme or color scheme?
input: What is my age ?
response: Your age is 12.
input: What is my age ?
response: Your age is 12.
input: What is my age ?
response: Your age is 12.

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: The party is a summer swimming pool party.
AI:

> Finished chain.


"That sounds like a fun event! For a summer swimming pool party, you'll want to wear something comfortable and appropriate for the water. A popular choice is a swimsuit or bikini paired with a stylish cover-up or sarong. You can also consider wearing a lightweight sundress or a tank top with shorts. Don't forget to bring a towel, sunscreen, and a pair of flip-flops or sandals for the poolside. Enjoy the party and have a great time!"

In [154]:
llmchain.predict(input="Suggest something else instead of tank top.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: The party is a summer swimming pool party.
response: That sounds like a fun event! For a summer swimming pool party, you'll want to wear something comfortable and appropriate for the water. A popular choice is a swimsuit or bikini paired with a stylish cover-up or sarong. You can also consider wearing a lightweight sundress or a tank top with shorts. Don't forget to bring a towel, sunscreen, and a pair of flip-flops or sandals for the poolside. Enjoy the party and have a great time!
input: Hi, Suggest me an outfit for a party.
response: Hello! I'd be happy to help you find an outfit for a party. Could you please provide me with some more info

"Sure! Instead of a tank top, you could consider wearing a lightweight blouse or a flowy tunic. These options will still keep you cool and comfortable for the summer swimming pool party. Another alternative could be a crop top paired with high-waisted shorts or a skirt. This will give you a trendy and stylish look while still being appropriate for the occasion. Don't forget to accessorize with some statement jewelry or a cute hat to complete your outfit!"

In [155]:
llmchain.predict(input="Alright, Summarize the final outfit then.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, Suggest me an outfit for a party.
response: Hello! I'd be happy to help you find an outfit for a party. Could you please provide me with some more information? What is the dress code for the party? Is there a specific theme or color scheme?
input: The party is a summer swimming pool party.
response: That sounds like a fun event! For a summer swimming pool party, you'll want to wear something comfortable and appropriate for the water. A popular choice is a swimsuit or bikini paired with a stylish cover-up or sarong. You can also consider wearing a lightweight sundress or a tank top with shorts. Don't forget to bring a towel, sunscreen, and

"Certainly! For your summer swimming pool party, the final outfit could be a stylish swimsuit or bikini paired with a lightweight blouse or flowy tunic. This will keep you cool and comfortable while still being appropriate for the occasion. You can also consider wearing high-waisted shorts or a skirt with a crop top for a trendy and stylish look. Don't forget to accessorize with statement jewelry or a cute hat to complete your outfit. And remember to bring a towel, sunscreen, and a pair of flip-flops or sandals for the poolside. Have a fantastic time at the party!"

In [157]:
llmchain.predict(input="Suggest me the earlier option instead of the blouse.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Suggest something else instead of tank top.
response: Sure! Instead of a tank top, you could consider wearing a lightweight blouse or a flowy tunic. These options will still keep you cool and comfortable for the summer swimming pool party. Another alternative could be a crop top paired with high-waisted shorts or a skirt. This will give you a trendy and stylish look while still being appropriate for the occasion. Don't forget to accessorize with some statement jewelry or a cute hat to complete your outfit!
input: Hi, Suggest me an outfit for a party.
response: Hello! I'd be happy to help you find an outfit for a party. Could you please provid

"Sure! Instead of the blouse, you could consider wearing a lightweight tunic or a flowy dress. These options will still keep you cool and comfortable for the summer swimming pool party. Another alternative could be a crop top paired with high-waisted shorts or a skirt. This will give you a trendy and stylish look while still being appropriate for the occasion. Don't forget to accessorize with some statement jewelry or a cute hat to complete your outfit!"